In [2]:
from datetime import datetime

# Get the current date and time
current_datetime = datetime.now()

print(current_datetime)

2024-12-13 18:33:37.367723


# Spatio Temporal Logistic
This notebook is the first test of a spatio temporal logistic to link properly the **built-up surface** of a given region depending on its quality of life (**GDP/cap**) and of its **population density**.

In [3]:
# import numpy as np
from Region import Region
from Df import Df
import pandas as pd
import geopandas as gpd
import os

In [4]:
from concurrent.futures import ThreadPoolExecutor
import itertools
from colorama import Fore, Style# init
from termcolor import colored

In [5]:
max_workers = 16

## PART 1 : Initialisation
Initialisation of the analysis parameters.

- **region_names** : (*string*) the country to study, named by their ISO3 
- **years** : (*string*) years to study
- **raster_S** : (*string*) letter used in the **GHSL** dataset (S, S_NRES, POP, ...)
- **lvl** : (*int*) the level of our administrative data (GDP and population)
- **subregion_borders** : (*string*) the path to administrative border shapefile to cut the subregions 
- **i dentifier** : (*string*) the column name to match the region names between the administrative data and the GIS data

To each region is associated a **DataFrame** (*oecd_DF_merged*) with the subregions matching administrative observables (GDP, population, etc).

In [6]:
lvl = 1

# GHSL type
raster_str = "Built_V"

In [7]:
data_folder = "/data/mineralogie/hautervo/data/"
folder_UCDB = data_folder + "Outputs/UCDB/"
folder_GHSL = data_folder + "Outputs/GHSL/"+raster_str+"/GADM_" + str(lvl) + "/"
folder_GHSL_POP = data_folder + "Outputs/GHSL/POP/GADM_" + str(lvl) + "/"
folder_DOSE = data_folder + "Outputs/DOSE/GADM_" + str(lvl) + "/"
folder_OSM_building = data_folder + "Outputs/OSM/building/GADM_" + str(lvl) + "/"

In [8]:
dose_df = pd.read_csv(data_folder+"DOSE/V2/DOSE_V2.9.csv")

In [22]:
# The OECD admin units

# oecd_admin_units = data_folder + "OECD/admin_units/TL" + str(lvl) + "/OECD_TL" + str(lvl) + "_2020_ESRI54009.shp"
# gpd_oecd_admin_units = gpd.read_file(oecd_admin_units)

gadm_admin_units = data_folder + "GADM/ESRI_54009/GADM_" + str(lvl) + "_ESRI54009.shp"
gpd_gadm_admin_units = gpd.read_file(gadm_admin_units, ignore_geometry=False)

excel_file = data_folder + "GHSL/UCDB/GHS_UCDB_GLOBE_R2024A.xlsx"

# Load all sheet names
# sheet_names = pd.ExcelFile(excel_file).sheet_names
# print(sheet_names)

In [10]:
UCDB_GHSL_df = pd.read_excel(excel_file, sheet_name="GHSL")
UCDB_SEC_df = pd.read_excel(excel_file, sheet_name="SOCIOECONOMIC")
UCDB_EMI_df = pd.read_excel(excel_file, sheet_name="EMISSIONS")
UCDB_CLI_df = pd.read_excel(excel_file, sheet_name="CLIMATE")
UCDB_INF_df = pd.read_excel(excel_file, sheet_name="INFRASTRUCTURES")

In [11]:
subregion_col = "GID_"+str(lvl)
parent_col = "GID_" + str(lvl-1) 

In [12]:
def convert_regions(region_list, admin_units, old_col, new_col):
    new_list = []
    for old_region in region_list:
        found = False
        for _, row in admin_units.iterrows():
            if row[old_col] == old_region and not found:
                new_list.append(row[new_col])
                found = True 
        if not found:
            print(old_region, " was not found.")
            new_list.append("")

    return new_list

In [13]:
# Countries to ignore from our study (not enough data)
# country_to_pop = ["SRB", "CRI", "ISR", "CYP", "ISL", "ALB", "LIE","MNE","MKD"]
country_to_pop = ["ATA"]

In [14]:
regions_names = list(gpd_gadm_admin_units["GID_0"].unique())

# #exclude some countries if necessary
for c in country_to_pop:
    try:
        regions_names.pop(regions_names.index(c))
    except:
        pass

# regions_names = ["FRA", "DEU", "GBR", "BEL", "ITA", "LUX", "ESP"] # to remove
# regions_names = ["FRA", "DEU", "GBR", "BEL", "ITA", "LUX", "ESP", "USA", "JPN", "CAN", "AUS"] # to remove
regions_names = ["FRA", "DEU", "GBR", "BEL", "ITA", "LUX", "ESP", "USA", "CAN", "AUS", "JPN", "CHN", "IND", "IDN"] # to remove
regions_names = ["FRA", "DEU", "GBR", "BEL", "ITA", "ESP", "USA", "CAN", "AUS", "JPN", "CHN", "IND", "RUS", "BRA", "ZAF", "IDN"] # to remove
regions_names = ["USA"]
# regions_names = convert_regions(regions_names, gpd_gadm_admin_units, "GID_0", "COUNTRY")

years = ["1975", "1990", "2000", "2010", "2020"] 
# years = ["2000", "2010", "2020"]
years = ["2010"]

regions = []

for name in regions_names:
    new_region = Region(name, lvl-1)
    regions.append(new_region)

    for y in years:      
        new_region.add_gis(data_folder + "GHSL/"+ raster_str + "/E" + y + "_100m_Global/subregions/" + name + ".tif", raster_str + "_" + y, str(y), lvl-1) 
        new_region.add_gis(data_folder + "GHSL/Built_POP/E" + y + "_100m_Global/subregions/" + name + ".tif", "POP_" + y, str(y), lvl-1) 

## PART 2 : Computation of the observables
Now that we define all the parameters of our study, we will cut the regions into their respective subregions (*make_subregions*). 

We then compute for each subregions GIS, the geographical observables that we store in its oecd_DF_merged.

In [15]:
def get_GHSL_values(subregion, folder: str, type: str, overwrite):    
    csv_path = os.path.join(folder, subregion.parent_name, subregion.name, '_'.join(years))+".csv"
    if not os.path.isfile(csv_path) or overwrite:
        # S
        if type == "S":
            output_df = pd.DataFrame({"year": years, "Built up surface GHSL":None, "Total surface":None, "Built up surface fraction":None})

            for y in years:
                gis = next((gis for gis in subregion.gis_list if gis.name == raster_str + "_" + y), None)
                if gis != None:
                    output_df.loc[output_df["year"]==y, "Built up surface GHSL"] = int(gis.get_total_sum_pixel_values())
                    output_df.loc[output_df["year"]==y, "Total surface"] = int(1e4*gis.get_total_number_pixels())
                    output_df.loc[output_df["year"]==y, "Built up surface fraction"] = output_df.loc[output_df["year"]==y, "Built up surface GHSL"] / output_df.loc[output_df["year"]==y, "Total surface"]
                else:
                    print(Fore.RED, f"{subregion.name} ", raster_str + "_" + y, " not found.", Style.RESET_ALL)

            # save the new df
            os.makedirs(os.path.dirname(csv_path), exist_ok=True)
            output_df.to_csv(csv_path, index=False)
            subregion.output_df_list.append(Df(output_df, type))
            print(colored(f"Saving {csv_path}", "green"))
        
        # V
        elif type == "V":
            output_df = pd.DataFrame({"year": years, "Built up volume GHSL":None, "Total surface":None,})

            for y in years:
                gis = next((gis for gis in subregion.gis_list if gis.name == raster_str + "_" + y), None)
                if gis != None:
                    output_df.loc[output_df["year"]==y, "Built up volume GHSL"] = int(gis.get_total_sum_pixel_values())
                    output_df.loc[output_df["year"]==y, "Total surface"] = int(1e4*gis.get_total_number_pixels())
                else:
                    print(Fore.RED, f"{subregion.name} ", raster_str + "_" + y, " not found.", Style.RESET_ALL)

            # save the new df
            os.makedirs(os.path.dirname(csv_path), exist_ok=True)
            output_df.to_csv(csv_path, index=False)
            subregion.output_df_list.append(Df(output_df, type))
            print(colored(f"Saving {csv_path}", "green"))

        # POP    
        elif type == "POP":
            output_df = pd.DataFrame({"year": years, "Population":None})

            for y in years:
                gis = next((gis for gis in subregion.gis_list if gis.name == "POP_" + y), None)
                if gis != None:
                    output_df.loc[output_df["year"]==y, "Population"] = int(gis.get_total_sum_pixel_values())
                else:
                    print(Fore.RED, f"{subregion.name} ", "POP_" + y, " not found.", Style.RESET_ALL)

            # save the new df
            os.makedirs(os.path.dirname(csv_path), exist_ok=True)
            output_df.to_csv(csv_path, index=False)
            subregion.output_df_list.append(Df(output_df, type))
            print(colored(f"Saving {csv_path}", "green"))

        # If we fall here, something wrong happened    
        else:
            print(f"Type of GHSL data to compute : {type} not understood.")
    else:
        print("Reading ", csv_path)
        #use a precomputed csv
        subregion.output_df_list.append(Df(pd.read_csv(csv_path), type))

In [16]:
def get_DOSE_values(subregion, folder: str, overwrite=False):
    csv_path = os.path.join(folder, subregion.parent_name, subregion.name, '_'.join(years))+".csv"

    if not os.path.isfile(csv_path) or overwrite:
        output_df = pd.DataFrame({"year": years, "grp_pc_usd_2015":None, "pop":None, "ag_grp_pc_usd_2015":None, "man_grp_pc_usd_2015":None, "serv_grp_pc_usd_2015":None, "PPP":None})

        for y in years:
            int_y = int(y)
            if y == "2020":
                int_y = 2018
                print("Using 2018 values")
            try:
                output_df.loc[output_df["year"]==y, "grp_pc_usd_2015"] = dose_df.loc[(dose_df["GID_1"] == subregion.name) & (dose_df["year"] == int_y), "grp_pc_usd_2015"].values[0]
            except Exception as e:
                print("1", e)

            try:
                output_df.loc[output_df["year"]==y, "pop"] = dose_df.loc[(dose_df["GID_1"] == subregion.name) & (dose_df["year"] == int_y), "pop"].values[0]
            except Exception as e:
                print("1 pop", e)

            try:
                output_df.loc[output_df["year"]==y, "ag_grp_pc_usd_2015"] = dose_df.loc[(dose_df["GID_1"] == subregion.name) & (dose_df["year"] == int_y), "ag_grp_pc_usd_2015"].values[0]
            except Exception as e:
                print("2",e)

            try:    
                output_df.loc[output_df["year"]==y, "man_grp_pc_usd_2015"] = dose_df.loc[(dose_df["GID_1"] == subregion.name) & (dose_df["year"] == int_y), "man_grp_pc_usd_2015"].values[0]
            except Exception as e:
                print("3", e)

            try:
                output_df.loc[output_df["year"]==y, "serv_grp_pc_usd_2015"] = dose_df.loc[(dose_df["GID_1"] == subregion.name) & (dose_df["year"] == int_y), "serv_grp_pc_usd_2015"].values[0]
            except Exception as e:
                print("4",e)

            try:
                output_df.loc[output_df["year"]==y, "PPP"] = dose_df.loc[(dose_df["GID_1"] == subregion.name) & (dose_df["year"] == int_y), "PPP"].values[0]
            except Exception as e:
                print("5",e)

        # save the new df
        os.makedirs(os.path.dirname(csv_path), exist_ok=True)
        output_df.to_csv(csv_path, index=False)
        subregion.output_df_list.append(Df(output_df, type))
        print(colored(f"Saving {csv_path}", "green"))
    else:
        #use a precomputed csv
        print("Reading ", csv_path)
        subregion.output_df_list.append(Df(pd.read_csv(csv_path), type))


In [17]:
def get_UCDB_values(subregion, folder: str, overwrite=False):
    csv_path = os.path.join(folder, subregion.parent_name, subregion.name, '_'.join(years))+".csv"

    if not os.path.isfile(csv_path) or overwrite:
        output_df = pd.DataFrame({"year": years, 
                                  "GH_BPC_TOT":None, 
                                  "GH_BPC_RES":None,
                                  "GH_BPC_NRE":None,
                                  "GH_POP_TOT":None, 
                                  "GH_BUS_TOT":None,
                                  "GH_BUS_RES":None,
                                  "GH_BUS_NRE":None,
                                  "GH_BUH_AVG":None,
                                  "GH_BUH_STD":None,
                                  "GH_BUV_TOT":None,
                                  "GH_BUV_RES":None,
                                  "GH_BUV_NRE":None,
                                  "GH_BUT_S11":None,
                                  "GH_BUT_S12":None,
                                  "GH_BUT_S13":None,
                                  "GH_BUT_S21":None,
                                  "GH_BUT_S22":None,
                                  "GH_BUT_S23":None,
                                  "GH_BUT_V11":None,
                                  "GH_BUT_V12":None,
                                  "GH_BUT_V13":None,
                                  "GH_BUT_V21":None,
                                  "GH_BUT_V22":None,
                                  "GH_BUT_V23":None,
                                  "GH_AGE_S75_2025":None,
                                  "GH_AGE_S85_2025":None,
                                  "GH_AGE_S95_2025":None,
                                  "GH_AGE_S05_2025":None,
                                  "GH_AGE_S15_2025":None,
                                  "GH_AGE_S25_2025":None,
                                  "SC_SEC_GDP":None, 
                                  "SC_SEC_GIF":None, 
                                  "SC_SEC_GIM":None,
                                  "EM_GHG_PEC":None,
                                  "EM_CO2_PEC":None,
                                  "EM_NOX_PEC":None,
                                  "EM_PM2_PEC":None,
                                  "EM_ENE_PER":None,
                                  "EM_RES_PER":None,
                                  "EM_IND_PER":None,
                                  "EM_TRA_PER":None,
                                  "EM_WAS_PER":None,
                                  "EM_AGR_PER":None,
                                  "CL_REN_PVO_2020":None,
                                  "CL_LCZ_A01_2025":None,
                                  "CL_LCZ_A02_2025":None,
                                  "CL_LCZ_A03_2025":None,
                                  "CL_LCZ_A04_2025":None,
                                  "CL_LCZ_A05_2025":None,
                                  "CL_LCZ_A06_2025":None,
                                  "CL_LCZ_A07_2025":None,
                                  "CL_LCZ_A08_2025":None,
                                  "CL_LCZ_A09_2025":None,
                                  "CL_LCZ_A10_2025":None,
                                  "CL_LCZ_A11_2025":None,
                                  "CL_LCZ_A12_2025":None,
                                  "CL_LCZ_A13_2025":None,
                                  "CL_LCZ_A14_2025":None,
                                  "CL_LCZ_A15_2025":None,
                                  "CL_LCZ_A16_2025":None,
                                  "CL_LCZ_A17_2025":None,
                                  "IN_ROA_LEN_2024":None,
                                  "IN_ROA_DEN_2024":None,
                                  "IN_CIS_ALL_2020":None,
                                  "IN_CIS_ENE_2020":None,
                                  "IN_CIS_TRA_2020":None,
                                  "IN_CIS_WAT_2020":None,
                                  "IN_CIS_WAS_2020":None,
                                  "IN_CIS_TEL_2020":None,
                                  "IN_CIS_HEA_2020":None,
                                  "IN_CIS_EDU_2020":None,
                                  })
        for col in output_df.columns.to_list():
            if col.startswith("GH") and not col.endswith("_2025"):
                for y in years:
                    try:
                        output_df.loc[output_df["year"]==y, col] = int(UCDB_GHSL_df.loc[UCDB_GHSL_df["GC_UCN_MAI_2025"] == subregion.name, col+"_"+y].values[0])
                    except Exception as e:
                        pass
                        # print(subregion.name, col, e)
            elif col.startswith("GH") and col.endswith("_2025"):
                try:
                    output_df.loc[output_df["year"]=="2020", col] = int(UCDB_GHSL_df.loc[UCDB_GHSL_df["GC_UCN_MAI_2025"] == subregion.name, col].values[0])
                except Exception as e:
                    print(subregion.name, col, e)
            elif col.startswith("SC"):
                for y in years:
                    try:
                        output_df.loc[output_df["year"]==y, col] = int(UCDB_SEC_df.loc[UCDB_SEC_df["GC_UCN_MAI_2025"] == subregion.name, col+"_"+y].values[0])
                    except Exception as e:
                        pass
                        # print(subregion.name, col, e)
            elif col.startswith("EM"):
                for y in years:
                    try:
                        output_df.loc[output_df["year"]==y, col] = float(UCDB_EMI_df.loc[UCDB_EMI_df["GC_UCN_MAI_2025"] == subregion.name, col+"_"+y].values[0])
                    except Exception as e:
                        pass
                        # print(subregion.name, col, e)
            elif col.startswith("CL"):
                try:
                    output_df.loc[output_df["year"]=="2020", col] = float(UCDB_CLI_df.loc[UCDB_CLI_df["GC_UCN_MAI_2025"] == subregion.name, col].values[0])
                except Exception as e:
                    pass
                    # print(subregion.name, col, e)
            elif col.startswith("IN"):
                try:
                    output_df.loc[output_df["year"]=="2020", col] = float(UCDB_INF_df.loc[UCDB_INF_df["GC_UCN_MAI_2025"] == subregion.name, col].values[0])
                except Exception as e:
                    print(subregion.name, col, e)


        # save the new df
        os.makedirs(os.path.dirname(csv_path), exist_ok=True)
        output_df.to_csv(csv_path, index=False)
        subregion.output_df_list.append(Df(output_df, type))
        print(colored(f"Saving {csv_path}", "green"))
    else:
        #use a precomputed csv
        print("Reading ", csv_path)
        subregion.output_df_list.append(Df(pd.read_csv(csv_path), type))


In [18]:
def get_OSM_areas(subregion, folder: str, overwrite=False): 
    gis_name = "OSM_building"  
    csv_path = os.path.join(folder, subregion.parent_name, subregion.name,  subregion.name +".csv")

    if not os.path.isfile(csv_path) or overwrite:
        gis = next((gis for gis in subregion.gis_list if gis.name == gis_name), None)
        if gis is not None:
            try:
                shp = gpd.read_file(gis.file)
                print("Starting area OSM ", gis.file)
                value = (shp["geometry"].area).sum()
                output_df = pd.DataFrame({gis_name+"_area":value}, index=[0])

                # save the new df
                os.makedirs(os.path.dirname(csv_path), exist_ok=True)
                output_df.to_csv(csv_path, index=False)
                print(colored(f"Saving {csv_path}", "green"))
                subregion.output_df_list.append(Df(output_df, gis_name))
                print("Ending area OSM ", gis.file)
            except Exception as e:
                print(e)
        else:
            print(f"OSM shp for {subregion.name} not found.")
    else:
        #use a precomputed csv
        print("Reading ", csv_path)
        subregion.output_df_list.append(Df(pd.read_csv(csv_path), gis_name))
    

### Preprocess with the desired computational methods

In [25]:
def preprocess():
    # Step 1 : make subregions
    overwrite = False

    print(Fore.GREEN + "Starting make_subregions()" + Style.RESET_ALL)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(executor.map(lambda region: region.make_subregions(gpd_gadm_admin_units, subregion_col, parent_col, overwrite=overwrite), regions))
        # list(executor.map(lambda region: region.make_subregions_ucdb(UCDB_GHSL_df, "GC_UCN_MAI_2025", "GC_CNT_GAD_2025", ), regions))

    # Step 2.1 : Computation
    overwrite = False

    subregions_list_parallel = []

    for region in regions:
        for subregion in region.subregions:
            subregions_list_parallel.append(subregion)
    #         try:
    #             file = os.path.join(folder_OSM_building, subregion.parent_name, subregion.name, subregion.name +".shp")
    #             if os.path.isfile(file):
    #                 subregion.add_gis(file, "OSM_building", "", "")
    #             else:
    #                 print(f"File {file} not found.")
    #         except Exception as e:
    #             print(e)
    # print(Fore.GREEN + "Starting UCDB" + Style.RESET_ALL)
    # with ThreadPoolExecutor(max_workers=max_workers) as executor:
    #     executor.map(get_UCDB_values, subregions_list_parallel, itertools.repeat(folder_UCDB), itertools.repeat(overwrite))

    # print(Fore.GREEN + "Starting GHSL_S" + Style.RESET_ALL)
    # with ThreadPoolExecutor(max_workers=max_workers) as executor:
    #     executor.map(get_GHSL_values, subregions_list_parallel, itertools.repeat(folder_GHSL), itertools.repeat("S"), itertools.repeat(overwrite))
    print(Fore.GREEN + "Starting GHSL_V" + Style.RESET_ALL)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(get_GHSL_values, subregions_list_parallel, itertools.repeat(folder_GHSL), itertools.repeat("V"), itertools.repeat(overwrite))
    print(Fore.GREEN + "Starting GHSL_POP" + Style.RESET_ALL)   
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(get_GHSL_values, subregions_list_parallel, itertools.repeat(folder_GHSL_POP), itertools.repeat("POP"), itertools.repeat(overwrite))
    print(Fore.GREEN + "Starting DOSE" + Style.RESET_ALL)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(get_DOSE_values, subregions_list_parallel, itertools.repeat(folder_DOSE), itertools.repeat(overwrite))

    # print(Fore.GREEN + "Starting OSM_area_computation()" + Style.RESET_ALL)
    # with ThreadPoolExecutor(max_workers=max_workers) as executor:
    #     executor.map(get_OSM_areas, subregions_list_parallel, itertools.repeat(folder_OSM_building), itertools.repeat(overwrite))

    del subregions_list_parallel # not needed anymore

In [26]:
preprocess()   

Starting make_subregions()
END: USA
Starting GHSL_V
Saving /data/mineralogie/hautervo/data/Outputs/GHSL/Built_V/GADM_1/USA/USA.9_1/2010.csv
Saving /data/mineralogie/hautervo/data/Outputs/GHSL/Built_V/GADM_1/USA/USA.8_1/2010.csv
Saving /data/mineralogie/hautervo/data/Outputs/GHSL/Built_V/GADM_1/USA/USA.7_1/2010.csv
Saving /data/mineralogie/hautervo/data/Outputs/GHSL/Built_V/GADM_1/USA/USA.3_1/2010.csv
Saving /data/mineralogie/hautervo/data/Outputs/GHSL/Built_V/GADM_1/USA/USA.16_1/2010.csv
Saving /data/mineralogie/hautervo/data/Outputs/GHSL/Built_V/GADM_1/USA/USA.11_1/2010.csv
Saving /data/mineralogie/hautervo/data/Outputs/GHSL/Built_V/GADM_1/USA/USA.12_1/2010.csv
Saving /data/mineralogie/hautervo/data/Outputs/GHSL/Built_V/GADM_1/USA/USA.1_1/2010.csv
Saving /data/mineralogie/hautervo/data/Outputs/GHSL/Built_V/GADM_1/USA/USA.14_1/2010.csv
Saving /data/mineralogie/hautervo/data/Outputs/GHSL/Built_V/GADM_1/USA/USA.19_1/2010.csv
Saving /data/mineralogie/hautervo/data/Outputs/GHSL/Built_V/GAD

In [ ]:
from datetime import datetime

# Get the current date and time
current_datetime = datetime.now()
print(Fore.GREEN, "Ended normally.", Style.RESET_ALL)
print(current_datetime)

 Ended normally. 
2024-12-13 16:03:48.711581


# TEST ZONE